# Wstęp do Sztucznej Inteligencji - rok akademicki 2020/2021

Przed rozpoczęciem pracy z notatnikiem zmień jego nazwę zgodnie z wzorem: `NrAlbumu_Nazwisko_Imie_PoprzedniaNazwa`.

Przed wysłaniem notatnika upewnij się, że rozwiązałeś wszystkie zadania/ćwiczenia.

# Temat: Optymalizacja globalna: Prosty algorytm genetyczny cz. II
Głownym celem zajęć poświęconych alorytmom genetycznym jest stworzenie od podstaw (implentacja) prostego algorytmu genetycznego i późniejsze wykorzystanie go do rozwiązania przykładowych zadań optymalizacji globalnej.

W tym notatniku będą Państwo mieli za zadanie zaimplementować trzy operatory genetyczne (trzy funkcje) wykonywane w głównej pętli algorytmu genetycznego. 

__Ważne:__ Wszystkie te funkcja wykonywane będą w każdej iteracji algorytmu należy zatem zadbać o to aby było ona zaimplementowana w sposób wydajny.

## Import biblioteki numpy
Wszystkie funkcje należy tworzyć z wykorzystaniem biblioteki `numpy`.

In [2]:
import numpy as np

## Przykładowa funkcja celu
Zadanie optymalizacji w wielu przypadakach sproawadza się optymalizacji odpowiednio sformułowanej funkcj, tzw. funkcji celu. Poniżej przykładowa prosta funkcja umożliwiająca testowanie zaimplemetowanych funkcji.  

In [5]:
# testowa funkcja celu
# x - jednowymiarowa tablica ndarray
def obj_func(x):
    return (x**2).sum()

Przykład wywołania.

Wektor `x` to tak zwany osobnik, czyli jedno z możliwych (dopuszczalnych) rozwiązań.  

In [6]:
# przykład wywołania
x = np.array([1.2, 0.1, 3, 2.1])
print(obj_func(x))

14.86


## Selekcja: Metoda koła ruletki
Metoda ruletki (koła ruletki)– jedna z najbardziej podstawowych metod selekcji. Polega na utworzeniu koła ruletki z polami odpowiadającymi poszczególnym osobnikom. Wielkość pól jest proporcjonalna do wartości funkcji przystosowania (celu). Proces selekcji oparty jest na obrocie ruletką tyle razy ile osobników jest w populacji i na wyborze za każdym razem jednego osobnika do nowej populacji. Pewne osobniki są wybierane więcej niż jeden raz, niektóre dokładnie raz, a niektóre wcale.

Koło ruletki w praktyce:
- Koło zestępujemy odcinkiem $[0,1]$, który dzielimy na tyle podprzedziałów ile osobników w populacji.
- Rozmiar każdego podprzedziału jest proporcjonalny do wartość funkcji celu danego osobnika, tzn. im większa wartość funkcji celu tym odcinek powinnien być większy. Ponieważ cały odcinek ma długośc 1 to rozmiary podprzedziałów można utożsamoć z prawdopodobieństwe przejścia osobnika do nowej populacji i policzyć ze wzoru $p_i= \frac{e_i}{\sum_i{e_i}}$, gdzie $i$ to numer osobnika, a $e_i$ to wartość funkcji celu $i$-tego osobnika.
- Zamiast kręcić kołem, losujemy liczbę z przedziału $[0,1]$ i sprawdzamy w którym podprzedziale się ona zawiera i osobnika o numerze zgodnym z numerem tego podprzedziału wybieramy do nowej populacji.  

Warto zwrócić na fakt, że tak zaproponowany wzór na długości przedziałów $p_i$ zadziała tylko wtedy gdy wszystkie wartości funkcji celu będą dodatnie. Aby to zapewnić wystarczy na początku do wektora wartości funkcji celu dodać odpowiednią stałą zapewniającą dodatniość (lub przynajmniej nieujemność) wszystkich elementów.



In [3]:
from math import fabs
def nbits(a, b, dx):
    ### TWÓJ KOD TUTAJ
    num = np.arange(a, b+dx, dx)
    B=0
    
    while(1):
        if 2**B > num.size:
            break
        else:
            B=B+1
            continue
    
    dx_new = (b-a)/((2**B) - 1)
    
    return B,dx_new

nbits(0,1,0.1) #widzimy bezbledne działanie funkcji





(4, 0.06666666666666667)

In [4]:
def gen_population(P, N, B):
    ### TWÓJ KOD TUTAJ
    ### Każdy wiersz to jeden osobnik, a więc będzie P wierszy Kolumn będzie N*B, bo każda zmienna jest zapisana na B-bitach
    pop = np.random.randint(2, size=(P, N*B))
    return pop
gen_population(1, 1, 5)

array([[1, 0, 1, 0, 0]])

In [5]:
def decode_individual(individual, N, B, a, dx):
    ### TWÓJ KOD TUTAJ
    temp = individual.reshape((N, B))
    wektor= np.zeros(B)
    k=1
    b_1=B
    for i in range (0,B):
        b_1=b_1-1
        wektor[i] = 2**(b_1)
        
        
    
    kk = 0
    kkk=0
    for i in range (0,B):
        kk=(wektor[i] * temp[0][i]) + kk
    for j in range (0,B):
        kkk=(wektor[j] * temp[1][j]) + kkk
       
    wynikowy = (kk * dx) + a
    wynikowy_1 = (kkk * dx) + a
    decode_individual = np.zeros(2)
    decode_individual[0] = wynikowy
    decode_individual[1] = wynikowy_1
        
    return decode_individual  

In [6]:
def evaluate_population(func, pop, N, B, a, dx):
    evaluated_pop = np.ndarray(int(pop.size/(B*N)))
    for i in range(int(pop.size/(B*N))):
        evaluated_pop[i] = func(decode_individual(pop[i], N, B, a, dx))
    return evaluated_pop
    


In [7]:
def get_best(pop, evaluated_pop):
    ### TWÓJ KOD TUTAJ
    best_value=evaluated_pop[0]
    for i in evaluated_pop:
        if i>best_value:   #i przyjmuje wartości kolejnych komórek tablicy tab
            best_value=i
    indeks=0    
    for j in range (0,evaluated_pop.size):
        if evaluated_pop[j] == best_value:
            indeks = j
    best_individual = np.zeros(pop.size)
    best_individual = pop[indeks]
    return best_individual, best_value

### Zadanie 1:
Zaimplementuj operator selekcji. Selekcja metodą koła ruletki.

Argumenty funkcji:
- `pop` - populacja zakodowanych osobników, tablica `ndarray`.
- `evaluated_pop` - tablica `ndarray` ocen osobników.
- `new_pop` - nowa populacja wybranych osobników (zakodowanych), tablica `ndarray`.

In [2]:
from __future__ import division
import numpy as np
import random,pdb
import operator

def roulette_selection_add(weights):
        '''performs weighted selection or roulette wheel selection on a list
        and returns the index selected from the list'''

        # sort the weights in ascending order
        sorted_indexed_weights = sorted(enumerate(weights), key=operator.itemgetter(1));
        indices, sorted_weights = zip(*sorted_indexed_weights);
        # calculate the cumulative probability
        tot_sum=sum(sorted_weights)
        prob = [x/tot_sum for x in sorted_weights]
        cum_prob=np.cumsum(prob)
        # select a random a number in the range [0,1]
        random_num=random.random()

        for index_value, cum_prob_value in zip(indices,cum_prob):
            if random_num < cum_prob_value:
                return index_value



In [164]:
def roulette(pop, evaluated_pop):
    ### TWÓJ KOD TUTAJ
    ### W moim przypadku wybieram tylko jednego osobnika 
    summm = evaluated_pop.sum()
    percentages = evaluated_pop/float(summm)
    indexx=roulette_selection_add(percentages) #funkcja roulette_selection_add jest nad naszą główną funkcją(roulette(...))
    new_pop = pop[indexx]
    
    return new_pop
testiner_pop= np.array([[0, 0, 1, 1, 1, 1, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 1, 0, 0, 1], [0, 1, 0, 0, 0, 1, 1, 1, 0, 0], [1, 0, 0, 1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 0, 1]])
test_epop  = np.array([0.30176899, 0.40998959, 0.88449532, 0.70135276, 1.75858481])
print("TEST:")
roulette(testiner_pop,test_epop)

TEST:


array([1, 1, 1, 1, 1, 1, 1, 1, 0, 1])

## Krzyżowanie
Operacja krzyżowania polega na wymianie fragmentów łańcuchów dwóch osobników rodzicielskich. Krzyżowanie jest kluczowym operatorem w algorytmach genetycznych, stanowiącym o ich sile i efektywności.

Ideą operatorów krzyżowania jest wymiana kodu genetycznego pomiędzy osobnikami, tak jak to się dzieje w naturze. Stworzono wiele teorii i rodzajów krzyżowań, które stosowane są do różnych rodzajów zadań i są zależne od sposobu kodowania. Dla potrzeb klasycznego algorytmu genetycznego opisano operator krzyżowania jednopunktowego. 

Krzyżowanie jednopunktowe – krzyżowanie zachodzi z pewnym prawdopodobieństwem `pk`; dla każdego osobnika losuje się liczbę i sprawdza, czy zachodzi krzyżowanie. Następnie dobiera się wybrane osobniki losowo w pary. Losuje się liczbę określającą miejsce krzyżowania i wymienia kod.

Podpowiedź: w praktyce krzyżowanie to można zaimplementować na dwa sposoby.

Sposób 1. Mając tablicę `pop` z parzystą liczbą osobników jako wiersze, dla każdych kolejnych dwóch wierszy wylosuj liczbę losową z przedziału `[0,1]`. Jeśli jest ona mniejsza niż prawdopodobieństwo krzyżowania, skrzyżuj osobniki i oboje potomków umieść w populacji wynikowej. Jeśli wartość losowa jest większa niż prawdopodobieństwo krzyżowania, skopiuj oboje rodziców do populacji wynikowej. 

Sposób 2. Mając tablicę `pop` z dowolną (niekoniecznie parzystą) liczbą osobników jako wiersze, dla każdego wiersza wylosuj liczbę losową z przedziału `[0,1]`. Jeśli jest ona mniejsza niż prawdopodobieństwo krzyżowania, skrzyżuj osobnika z osobnikiem w następnym wierszu i dowolnego z potomków (jednego!) umieść w populacji wynikowej. Jeśli wartość losowa jest większa niż prawdopodobieństwo krzyżowania, skopiuj aktualny wiersz do populacji wynikowej. Uwaga: ostatni wiersz krzyżowany jest z pierwszym (następnym w sensie "modulo").

![krzyzowanie.PNG](http://torus.uck.pk.edu.pl/~amarsz/images/krzyzowanie.PNG)




### Zadanie 2
Zaimplementuj operator krzyżowania. Krzyżowanie jedno-punktowe.

Argumenty funkcji:
- `pop` - populacja zakodowanych osobników, tablica `ndarray`.
- `pk` - prawdopodobieństwo krzyżowania dla pary osobników. Liczba rzeczywista z przedziału $[0,1]$.
- `new_pop` - nowa populacja osobników po krzyżowaniu (zakodowanych), tablica `ndarray`.

In [150]:
import copy
def cross_helper(a,b,c): #supporter_for_cross_main_funct
    
    aa = copy.copy(a)
    bb=copy.copy(b)
    sizer = len(aa)
    aa[c:sizer] =  bb[c:sizer]
    return aa

def cross(pop, pk):
    ### TWÓJ KOD TUTAJ
    sizeeer = len(pop[0]) #rozmiar kazdego wiersza
    sizeeer_il_wierszy = len(pop)
    new_pop = np.zeros((sizeeer_il_wierszy,sizeeer))
    
    cross_pointer = random.randint(0, len(pop[0]))
    for i in range (0,sizeeer_il_wierszy-1):
        a = random.random()
        
        if (a > pk):
            new_pop[i] = pop[i]
           
        else:
            new_pop[i] = cross_helper(pop[i],pop[i+1],cross_pointer)
            
        
     
    random_a_2 = random.random()
    if random_a_2 < pk :
        new_pop[sizeeer_il_wierszy-1] = cross_helper(pop[sizeeer_il_wierszy-1],pop[0],cross_pointer)
    else:
        new_pop[sizeeer_il_wierszy-1] = pop[sizeeer_il_wierszy-1]
     
    
    return new_pop
    
Test= np.array([[0, 0, 1, 1, 1, 1, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 1, 0, 0, 1], [0, 1, 0, 0, 0, 1, 1, 1, 0, 0], [1, 0, 0, 1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 0, 1]])
print("Test:")
cross(Test, 0.6)

Test:


array([[0., 0., 1., 1., 1., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 1., 1., 1., 0., 0.],
       [0., 1., 0., 0., 0., 1., 1., 1., 0., 0.],
       [1., 0., 0., 1., 1., 1., 1., 1., 0., 1.],
       [1., 1., 1., 1., 1., 1., 0., 0., 0., 0.]])

## Mutacja
Mutacja polega na wprowadzeniu do istniejących, zakodowanych osobników, pewnych losowych zmian. Mutacja tworzy nowego osobnika na bazie jednego i tylko jednego rodzica. Jest wiele metod tworzenia nowych osobników za pomocą operatora mutacji.

Podstawową formę mutacji można zapisać następująco:
$$x'= m(x)$$
gdzie: $x$ jest osobnikiem rodzica, $m$ funkcją mutacji, $x'$ to potomek, osobnik zmutowany.

Mutacja również zachodzi z pewnym zadanym prawdopodobieństwem `pm`. Można ją zaimplementować na dwa sposoby.

Sposób 1. Dla każdego osobnika losuje się liczbę sprawdzając czy będzie on podlegał mutacji. Jeśli tak to losuje się gen, który będzie zmutowany i dokonuje się mutacji (tu: negacja bitu). 

Sposób 2. Dla danego osobnika, dla każdego genu (bitu) osobno losuje się liczbę sprawdzającą czy będzie podlegał mutacji. Jeśli tak, dokonuje się jego mutacji (u nas: negacja bitu).

Mutacja w klasycznym algorytmie genetycznym odgrywa drugorzędną rolę. Częstość mutacji potrzebna do uzyskania dobrych wyników w empirycznych badaniach nad algorytmami genetycznymi jest rzędu jeden do tysiąca skopiowanych bitów. W naturalnych populacjach częstość jest równie mała lub nawet mniejsza. 

W przypadku osobników zakodowanych binarnie (klasyczny algorytm genetyczny) nie ma problemu ze stosowaniem mutacji (po prostu zamieniamy wartość genu na przeciwny). 

### Zadanie 3
Zaimplementuj operator mutacji. Mutacja bitowa.

Argumenty funkcji:
- `pop` - populacja zakodowanych osobników, tablica `ndarray`.
- `pm` prawdopodobieństwo mutacji dla pojedynczego bitu. Liczba rzeczywista z przedziału $[0,1]$.
- `new_pop` - nowa populacja osobników po mutacji (zakodowanych), tablica `ndarray`.

In [151]:
def mutate(pop, pm):
    ### TWÓJ KOD TUTAJ
    for swapped in range(len(pop)):
        if(random.random() < pm):
            swapWith = int(random.random() * len(pop))
            
            city1 = pop[swapped]
            city2 = pop[swapWith]
            
            pop[swapped] = city2
            pop[swapWith] = city1
    new_pop = pop
    return new_pop
#test
popss = np.array([[0, 0, 1, 1, 1, 1, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 1, 0, 0, 1], [0, 1, 0, 0, 0, 1, 1, 1, 0, 0], [1, 0, 0, 1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 0, 1]])
print("TEST:")
mutate(popss,0.05)

TEST:


array([[0, 0, 1, 1, 1, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 1, 0, 0, 0, 1, 1, 1, 0, 0],
       [1, 0, 0, 1, 1, 1, 1, 1, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 1]])

&copy; Katedra Informatyki, Politechnika Krakowska